# News Preprocessing - Load dữ liệu từ các file CSV

## 1. Import và các hàm hỗ trợ

### 1.1. Các thư viện hỗ trợ
1. ```urlmarker```: Tham khảo tại [Url extraction in python - Ryan Compton](http://ryancompton.net/2015/02/16/url-extraction-in-python/)
2. ```token_sylabling```: Tham khảo tại: [core_nlp/tokenization/base_tokenizer.py](https://github.com/deepai-solutions/core_nlp/blob/master/tokenization/base_tokenizer.py)

In [1]:
import json, os, ast, re, sys
import csv
import unicodedata as ud
import random

from text_utils import remove_stopwords, token_sylabling
from utils import load_file_with_newline
import urlmarker

### 1.2. Các đường dẫn đến các file dataset

Thư mục _Data_ chứa các file trích xuất từ dataset ___[VFND](https://github.com/thanhhocse96/vfnd-vietnamese-fake-news-datasets)___

In [2]:
data_path = './Data/'

Một số file CSV được trích xuất từ ___VFND___:
1. [vn_news_226_tlfr.csv](): Tin tức và Facebook post, 2 trường: _text_ (Với các bài báo là Tiêu đề + Nội dung, với Facebook post là nội dung của post đó) và _label_

In [3]:
csv_text_label_226 = 'vn_news_226_tlfr.csv'

### 1.3. Đường dẫn đến các file từ điển

In [4]:
dict_path = './Dictionaries'

bi_gram_path = os.path.join(dict_path, 'bi_gram.txt')
tri_gram_path = os.path.join(dict_path, 'tri_gram.txt')
four_gram_path = os.path.join(dict_path, 'four_gram.txt')

stopword_path = os.path.join(dict_path, 'Stopwords_vi.txt')

Load các từ điển đó lên và chứa vào các Set() dữ liệu

In [5]:
bi_dict = load_file_with_newline(bi_gram_path)
tri_dict = load_file_with_newline(tri_gram_path)
four_dict = load_file_with_newline(four_gram_path)

stopwords = load_file_with_newline(stopword_path)

Đưa các từ trong từ điển về dạng chữ thường

In [6]:
bi_gram = [x.lower() for x in bi_dict]
tri_gram = [x.lower() for x in tri_dict]
four_gram = [x.lower() for x in four_dict]

### 1.4. Thuật toán Longest Matching 
Thuật toán ___Longest Matching___ gộp các syllable token - tiếng - thành các từ được định nghĩa trong bộ từ điển. Input: 
1. ```token```: List của các syllable sau khi tách ra
2. ```bi_gram, tri_gram, four_gram```: Các set() từ điển đã load ở trên

In [7]:
def LongestMatching(token, bi_gram, tri_gram, four_gram):
    #token_len: Length of syllable list
    token_len = len(token)
    cur_id = 0
    word_list = []
    
    #done: True when cur_id reach 
    done = False
    
    while (cur_id < token_len) and (not done):
        cur_word = token[cur_id]
        if(cur_id >= token_len - 1):
            word_list.append(cur_word)
            done = True
        else:
            next_word = token[cur_id + 1]
            bi_word = " ".join([cur_word.lower(), next_word.lower()])
            if(cur_id >= token_len - 2):
                if bi_word in bi_gram:
                    word_list.append("_".join([cur_word, next_word]))
                    cur_id  = cur_id + 2
                else: 
                    word_list.append(cur_word)
                    cur_id  = cur_id + 1
                        
            else: 
                bi_next_word = token[cur_id + 2]
                tri_word = " ".join([bi_word, bi_next_word.lower()])
                if(cur_id >= token_len - 3):
                    if tri_word in tri_gram:
                        word_list.append("_".join([cur_word, next_word, bi_next_word]))
                        cur_id  = cur_id + 3
                    elif bi_word in bi_gram:
                        word_list.append("_".join([cur_word, next_word]))
                        cur_id = cur_id + 2
                    else:
                        word_list.append(cur_word)
                        cur_id = cur_id + 1
                else:
                    tri_next_word = token[cur_id + 3]
                    four_word = " ".join([tri_word, tri_next_word.lower()])
                    if four_word in four_gram:
                        word_list.append("_".join([cur_word, next_word, bi_next_word, tri_next_word]))
                        cur_id  = cur_id + 4
                    elif tri_word in tri_gram:
                        word_list.append("_".join([cur_word, next_word, bi_next_word]))
                        cur_id  = cur_id + 3
                    elif bi_word in bi_gram:
                        word_list.append("_".join([cur_word, next_word]))
                        cur_id = cur_id + 2
                    else:
                        word_list.append(cur_word)
                        cur_id = cur_id + 1
    return word_list

### 1.5. Text Preprocessing
Input: văn bản tin tức => Output: Văn bản của các cụm từ đã được gom cụm theo bộ từ điển:
1. Chuyển đổi text thành các âm tiết bằng hàm: _token_\__sylabling_ 
2. Dùng longest matching để chuyển các tiếng thành các từ dựa theo bộ từ điển
3. Loại bỏ stopword

In [8]:
def text_preprocessing(text):
    token_list = token_sylabling(text)
    word_list = LongestMatching(token_list, bi_gram, tri_gram, four_gram)
    remove_stopword_list = remove_stopwords(word_list, stopwords)
    new_text = ' '.join(remove_stopword_list)
    return new_text

## 2. Load và xử lý các file

### 2.1 Load và xử lý ```vn_news_226_tlfr.csv```

Load ```vn_news_226_tlfr.csv``` vào List các Dictionary

In [9]:
newslist_1 = []

with open(os.path.join(data_path, csv_text_label_226), newline='', encoding = 'UTF-8') as csv_file:
    reader = csv.DictReader(csv_file)
    newslist_1 = list(reader)

List các tin tức đã được tiền xử lý theo mục [1.5]

In [10]:
preprocessed_news_1 = []
for news in newslist_1:
    pre_news = dict()
    pre_news['text'] = text_preprocessing(news['text'])
    pre_news['label'] = news['label']
    preprocessed_news_1.append(pre_news)

## 3. Xuất các kết quả tiền xử lý ra file CSV

In [11]:
preproc_path = './PreprocessingData'

### 3.1. Xuất kết quả tiền xử lý ```vn_news_226_tlfr.csv``` ra ```preproc_vn_news_226_tlfr.csv```
Shuffle thứ tự các tin tức trong ```preprocessed_news_1```, tham khảo tại: [Shuffling a list of objects](https://stackoverflow.com/a/976918/5144980).

In [12]:
shuffle_id_list = [i for i in range(len(preprocessed_news_1))]
random.shuffle(shuffle_id_list)

Đưa kết quả preprocessing ra file ```preproc_vn_news_226_tlfr.csv```. Tham khảo thêm tại:
1. [How do I write a Python dictionary to a csv file? [duplicate]](https://stackoverflow.com/a/10373268/5144980)

In [13]:
keys = preprocessed_news_1[0].keys()
print(keys)
print(type(preprocessed_news_1[0]))

dict_keys(['text', 'label'])
<class 'dict'>


In [14]:
with open(os.path.join(preproc_path, 'preproc_vn_news_226_tlfr.csv'), 'w') as csv_file:
    dict_writer = csv.DictWriter(csv_file, fieldnames = keys)
    dict_writer.writeheader()
    for i in range(len(preprocessed_news_1)):
        dict_writer.writerow(preprocessed_news_1[shuffle_id_list[i]])

## Thử nghiệm

In [15]:
text = "ngày 21/11/2018, Đây là game GARRY'S MOD: tuyệt vô âm tín ==> => chơi trên kênh DR.Trực Tiếp Game test word: tu nhân tích đức  tructiepgame@gmail.com.vn của tôi - Dũng CT. tp.HCM, TP.HCM Rất mong nhận được sự ủng hộ của ae. ++ Đăng ký theo dõi kênh tại đây: https://goo.gl/A7BCZV ++ LINK TẢI APP CUBETV: https://www.cubetv.sg ++ THEO DÕI TTG TRÊN CUBETV: http://www.cubetv.sg/18947372 ++ DONATE ĐỂ TTG MUA ĐƯỢC NHIỀU GAME HƠN TẠI: - https://playerduo.com/tructiepgame - https://streamlabs.com/tructiepgamevn ++ MUA/THUÊ GAME BẢN QUYỀN GIÁ RẺ TẠI: Website: http://divineshop.vn Fanpage: https://www.facebook.com/Divine.Shop...., da che mắt ngựa"
token_list = token_sylabling(text)
print(token_list)

['ngày', '21/11/2018', ',', 'Đây', 'là', 'game', 'GARRY', "'", 'S', 'MOD', ':', 'tuyệt', 'vô', 'âm', 'tín', '==>', '=>', 'chơi', 'trên', 'kênh', 'DR.', 'Trực', 'Tiếp', 'Game', 'test', 'word', ':', 'tu', 'nhân', 'tích', 'đức', 'tructiepgame@gmail.com.vn', 'của', 'tôi', '-', 'Dũng', 'CT.', 'tp.', 'HCM', ',', 'TP.', 'HCM', 'Rất', 'mong', 'nhận', 'được', 'sự', 'ủng', 'hộ', 'của', 'ae.', '+', '+', 'Đăng', 'ký', 'theo', 'dõi', 'kênh', 'tại', 'đây', ':', 'https://goo.gl/A7BCZV', '+', '+', 'LINK', 'TẢI', 'APP', 'CUBETV', ':', 'https://www.cubetv.sg', '+', '+', 'THEO', 'DÕI', 'TTG', 'TRÊN', 'CUBETV', ':', 'http://www.cubetv.sg/18947372', '+', '+', 'DONATE', 'ĐỂ', 'TTG', 'MUA', 'ĐƯỢC', 'NHIỀU', 'GAME', 'HƠN', 'TẠI', ':', '-', 'https://playerduo.com/tructiepgame', '-', 'https://streamlabs.com/tructiepgamevn', '+', '+', 'MUA', '/', 'THUÊ', 'GAME', 'BẢN', 'QUYỀN', 'GIÁ', 'RẺ', 'TẠI', ':', 'Website', ':', 'http://divineshop.vn', 'Fanpage', ':', 'https://www.facebook.com/Divine.Shop', '...', '.', ','

In [16]:
token_list_1 = LongestMatching(token_list, bi_gram, tri_gram, four_gram)
print(token_list_1)

['ngày', '21/11/2018', ',', 'Đây', 'là', 'game', 'GARRY', "'", 'S', 'MOD', ':', 'tuyệt_vô_âm_tín', '==>', '=>', 'chơi', 'trên', 'kênh', 'DR.', 'Trực_Tiếp', 'Game', 'test', 'word', ':', 'tu_nhân_tích_đức', 'tructiepgame@gmail.com.vn', 'của', 'tôi', '-', 'Dũng', 'CT.', 'tp.', 'HCM', ',', 'TP.', 'HCM', 'Rất', 'mong', 'nhận', 'được', 'sự', 'ủng_hộ', 'của', 'ae.', '+', '+', 'Đăng_ký', 'theo_dõi', 'kênh', 'tại', 'đây', ':', 'https://goo.gl/A7BCZV', '+', '+', 'LINK', 'TẢI', 'APP', 'CUBETV', ':', 'https://www.cubetv.sg', '+', '+', 'THEO_DÕI', 'TTG', 'TRÊN', 'CUBETV', ':', 'http://www.cubetv.sg/18947372', '+', '+', 'DONATE', 'ĐỂ', 'TTG', 'MUA', 'ĐƯỢC', 'NHIỀU', 'GAME', 'HƠN', 'TẠI', ':', '-', 'https://playerduo.com/tructiepgame', '-', 'https://streamlabs.com/tructiepgamevn', '+', '+', 'MUA', '/', 'THUÊ', 'GAME', 'BẢN_QUYỀN', 'GIÁ', 'RẺ', 'TẠI', ':', 'Website', ':', 'http://divineshop.vn', 'Fanpage', ':', 'https://www.facebook.com/Divine.Shop', '...', '.', ',', 'da_che_mắt_ngựa']


In [17]:
preprocessing_text = text_preprocessing(text)
print(preprocessing_text)

ngày 21/11/2018 , Đây game GARRY ' S MOD : tuyệt_vô_âm_tín ==> => chơi kênh DR. Trực_Tiếp Game test word : tu_nhân_tích_đức tructiepgame@gmail.com.vn tôi - Dũng CT. tp. HCM , TP. HCM Rất mong nhận ủng_hộ ae. + + Đăng_ký theo_dõi kênh : https://goo.gl/A7BCZV + + LINK TẢI APP CUBETV : https://www.cubetv.sg + + THEO_DÕI TTG TRÊN CUBETV : http://www.cubetv.sg/18947372 + + DONATE ĐỂ TTG MUA ĐƯỢC NHIỀU GAME HƠN TẠI : - https://playerduo.com/tructiepgame - https://streamlabs.com/tructiepgamevn + + MUA / THUÊ GAME BẢN_QUYỀN GIÁ RẺ TẠI : Website : http://divineshop.vn Fanpage : https://www.facebook.com/Divine.Shop ... . , da_che_mắt_ngựa
